## Verificar Erros no CSV

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import tensorflow as tf
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


In [2]:
cols = ["Começou", "Jogada 1", "Jogada 2","Jogada 3", "Jogada 4", "Jogada 5", "Jogada 6",
        "Jogada 7", "Jogada 8","Jogada 9",'M00','M01','M02','M10','M11','M12','M20','M21','M22',"Terminou na Rodada","Ganhou"]     

tabuleiro_cols = ['M00','M01','M02','M10','M11','M12','M20','M21','M22']

jogadas_list_cols = ["Jogada 1", "Jogada 2","Jogada 3", "Jogada 4", "Jogada 5", "Jogada 6",
                "Jogada 7", "Jogada 8","Jogada 9"]
        
data =  pd.read_csv('jogadas.csv',names=cols)

In [3]:
len(data)

22604

In [4]:
## Verifica erros de duplicidade de informação, ou seja, linha x coluna y sendo usadas mais de uma vez no mesmo jogo

lista_erro_dupl_index = []
erro_string = []
quem_comecou = []
dupl = []
rodadaaa = []
jogadas = ['Jogada 1', 'Jogada 2', 'Jogada 3', 'Jogada 4', 'Jogada 5',
       'Jogada 6', 'Jogada 7', 'Jogada 8', 'Jogada 9']
verify_error = data.copy()
dicionario = {'Player': 0,'Computer':1}


for i in range(len(verify_error)):
    comp = -1
    cont = 0
    for col in jogadas:
        value = verify_error.loc[i,col]
        if (value == comp) and (value != '0'):
            cont +=1
            lista_erro_dupl_index.append(i)
            erro_string.append(f"VALUE: {value}| COMP: {comp}")
            if (verify_error.loc[i,'Começou'] == dicionario['Computer']) and (int(col[-1])%2 != 0):
                quem_comecou.append(f'Computador Error C')
            elif (verify_error.loc[i,'Começou'] == dicionario['Computer']) and (int(col[-1])%2 == 0):
                quem_comecou.append(f'Player Error C')
            elif (verify_error.loc[i,'Começou'] == dicionario['Player']) and (int(col[-1])%2 != 0):
                quem_comecou.append(f'Player Error P')
            elif (verify_error.loc[i,'Começou'] == dicionario['Player']) and (int(col[-1])%2 == 0):
                quem_comecou.append(f'Computer Error P')
            rodadaaa.append(col)
            break
        comp = value
            
    if cont > 1:
        dupl.append('Linha {i}, {cont} Vezes')
            
print(lista_erro_dupl_index)
        
    

[]


In [5]:
def check_if_win(states):
    """Check if there's a winner or a tie."""
    for i in range(3):
        # Check rows
        if states[i][0] == states[i][1] == states[i][2] != 0:
            return ['Ganhou',states[i][0]]

        # Check columns
        if states[0][i] == states[1][i] == states[2][i] != 0:
            return ['Ganhou',states[0][i]]

    # Check diagonals
    if states[0][0] == states[1][1] == states[2][2] != 0:
        return ['Ganhou',states[1][1]]
    
    if states[0][2] == states[1][1] == states[2][0] != 0:
        return ['Ganhou',states[1][1]]

    # Check for tie (if all spots are filled and no winner)
    if all(cell != 0 for row in states for cell in row):
        return ['Empate','']
    return ['','']


In [6]:
# Check if the Win Column and the finish column is right 
# If the game result is correct
states = [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ]
problem_list_index_winner = []
problem_list_index_states = []
check_win_data = data.copy()
for i in range(len(check_win_data)):
    count_rodada = 0
    states = [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ]
    vencedor = []
    #print(i)
    for col in tabuleiro_cols:
        value = check_win_data.loc[i,col]
        #print(f'Posição {col}, Valor -> {value}')
        #print(int(col[1]),int(col[-1]))
        if value == "Computer Played":
            states[int(col[1])][int(col[-1])] = 'X'
            count_rodada += 1
            if check_if_win(states)[0] == 'Ganhou' and check_if_win(states)[1] == 'X':
                vencedor.append(check_if_win(states)[0])
            elif check_if_win(states)[0] == 'Empate':
                vencedor.append(check_if_win(states)[0])
        elif value == "Opponent Played":
            states[int(col[1])][int(col[-1])]= 'O'
            count_rodada += 1
            if ((check_if_win(states)[0] == 'Ganhou') and (check_if_win(states)[1] == 'O')):
                vencedor.append('Perdeu')
            elif check_if_win(states)[0] == 'Empate':
                vencedor.append(check_if_win(states)[0])
    vencedor = list(dict.fromkeys(vencedor))

    if vencedor != []:    
        for win in vencedor:
            if (win != check_win_data.loc[i,'Ganhou']):
                problem_list_index_winner.append(i)
                problem_list_index_states.append(states)
    else:
        problem_list_index_winner.append(i)
        problem_list_index_states.append(states)
                

    
        #states[value[1:]]
    
problem_list_index_winner


[]

In [7]:
# Verificar se a matrix esta correta
check_pos = data.copy()

who_played = ['Computer Played', 'Opponent Played']
number_player = {'Player': 0,'Computer':1}

jogadas = ['Jogada 1', 'Jogada 2', 'Jogada 3', 'Jogada 4', 'Jogada 5',
       'Jogada 6', 'Jogada 7', 'Jogada 8', 'Jogada 9']

problem_list_index_matrix = []
for i in range(len(check_pos)):
    who = ''
    error_matriz = []
    if check_pos.loc[i,'Começou'] == number_player['Computer']:
        who = who_played[0]
    elif check_pos.loc[i,'Começou'] == number_player['Player']:
        who = who_played[1]        
    for col in jogadas:
        value = check_pos.loc[i,col]
        if value != '0':
            string = f'M{value[7]}{value[-1]}'
            if check_pos.loc[i,string] != who:
                error_matriz.append(f'Posição {value} valor {who}, na tabela esta {string}')
        elif value == '0':
            break
        who = who_played[1] if who == who_played[0] else who_played[0]
            #print(i,col,who)
    if error_matriz != []:
        problem_list_index_matrix.append(i)
    



problem_list_index_matrix



[]

In [8]:
print(f'Indexes com problemas: \nDuplicidade de Valores nas células:\n{lista_erro_dupl_index}\nProblema de Erro na avaliação da vitória\n{problem_list_index_winner}')
print(f'Problema na construção da matrix de jogo\n{problem_list_index_matrix}')

Indexes com problemas: 
Duplicidade de Valores nas células:
[]
Problema de Erro na avaliação da vitória
[]
Problema na construção da matrix de jogo
[]


In [9]:
do =  input('Would you like to pop the data with problem?\nYes or No')

if do=='Yes':
    to_pop = lista_erro_dupl_index + problem_list_index_winner + problem_list_index_matrix
    to_pop = list(dict.fromkeys(to_pop))
    data.drop(to_pop,axis='index',inplace=True)
    data.reset_index(drop=True,inplace=True)


In [10]:
# Se precisar, caso não deixar comentado
# data.to_csv('jogadas.csv',columns=cols,index=False,header=False)

In [11]:
len(data)

22604

In [12]:
## KNN model
# Tirando as colunas das jogadas seguintes o resultado piora
# Retirando as colunas que coloqui posteriormente de posição na matriz o resultado piora
# 'M00','M01','M02','M10','M11','M12','M20','M21','M22',"Terminou na Rodada"
# filtrando apenas pelas vitórias, o que teoricamente poderia ajudar a apenas ter vitórias não ajuda, mas dependendo do numero de vizinhos pode ajudar
# https://www.statology.org/sklearn-classification-report/
#for i in range(9):
#    jogada_a_analisar = f'Jogada {i+1}'
#    numero = int(jogada_a_analisar[-1])
#    colunas_para_tirar = []#jogadas_list_cols[numero:]
#    #test_lb_encoder = test_lb_encoder[test_lb_encoder['Ganhou']==map_column_ganhou['Ganhou']]
#    train_lb_array, X_train_lb_encoder, y_train_lb_encoder = scale_dataset(train_lb_encoder[train_lb_encoder.columns.drop(colunas_para_tirar)],jogada_a_analisar,oversample=True)
#    test_lb_array, X_test_lb_encoder, y_test_lb_encoder = scale_dataset(test_lb_encoder[test_lb_encoder.columns.drop(colunas_para_tirar)],jogada_a_analisar,oversample=False)
#    
#    knn_model = KNeighborsClassifier(n_neighbors=3)
#    knn_model.fit(X_train_lb_encoder,y_train_lb_encoder)
#    y_pred = knn_model.predict(X_test_lb_encoder)
#    #print(y_test_lb_encoder,y_pred)
#    print(f'Rodada Analisada {i+1}')
#    print(classification_report(y_test_lb_encoder,y_pred,zero_division=0))
#    #classification_report(y_test_lb_encoder,y_pred)